<a href="https://colab.research.google.com/github/sgywhetstone/CholletDLWorkbooks/blob/master/CholletC3IMBDClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a simple binary classification task taken from C3.4 of Chollet. This network classifies IMDB movie reviews into 'positive' and 'negative' categories. Training is done on 25k reviews taken from the Keras IMBD dataset.


In [16]:
import keras
import numpy as np
keras.__version__

'2.2.4'

In [0]:
#Set the number of words we keep from the reviews - e.g. if max_words = 10000, we keep only the top 10 000 most common words
max_words = 10000

7/3/19 - Change in numpy leads to book implementation failing to load data. Fix from https://stackoverflow.com/a/56243777 

Orginal implementation retains only lines 1 and 10. 

In [0]:
from keras.datasets import imdb

# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words= max_words)

# restore np.load for future normal usage
np.load = np_load_old

In [0]:
#This decodes a selected review
#word_index is a dictionary (word, index)
word_index = imdb.get_word_index()
#flip the dictionary
reverse_word_index = dict(
  [(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join(
[reverse_word_index.get(i-3, '?') for i in train_data[24000]])

In [0]:
#Encode the reviews via one-hot encoding
def vectorize_sequences(sequences, dimension = max_words):  #One-hot function
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
    results[i, sequence] = 1.
  return results

#Vectorize training and test data
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

#Vectorize training and test labels
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

We build a 3 layer network via Sequential - Two 16-dimensional relu layers, and one sigmoid layer to output probability. 

In [0]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [36]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

W0704 00:18:24.492415 140302959847296 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0704 00:18:24.522816 140302959847296 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0704 00:18:24.532229 140302959847296 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
from keras import optimizer